In [1]:
import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random as python_random
import pickle
import keras

import tensorflow as tf
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers


2023-03-14 15:02:04.476713: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
aggregated_data = pickle.load( open( '../data/processed/20230304/aggregated_data', 'rb' ) )
[train_mps_raw_pca, valid_mps_raw_pca, test_mps_raw_pca,
    train_mps_bg_pca, valid_mps_bg_pca, test_mps_bg_pca,
    train_mps_fg_pca, valid_mps_fg_pca, test_mps_fg_pca,
    train_indices_raw_pca, valid_indices_raw_pca, test_indices_raw_pca,
    train_indices_bg_pca, valid_indices_bg_pca, test_indices_bg_pca,
    train_indices_fg_pca, valid_indices_fg_pca, test_indices_fg_pca,
    train_embedding_raw_pca, valid_embedding_raw_pca, test_embedding_raw_pca,
    train_embedding_bg_pca, valid_embedding_bg_pca, test_embedding_bg_pca,
    train_embedding_fg_pca, valid_embedding_fg_pca, test_embedding_fg_pca,
    train_vgg_raw_pca, valid_vgg_raw_pca, test_vgg_raw_pca,
    train_vgg_bg_pca, valid_vgg_bg_pca, test_vgg_bg_pca,
    train_vgg_fg_pca, valid_vgg_fg_pca, test_vgg_fg_pca,
    train_panns_clip_raw_pca, valid_panns_clip_raw_pca, test_panns_clip_raw_pca,
    train_panns_clip_bg_pca, valid_panns_clip_bg_pca, test_panns_clip_bg_pca,
    train_panns_clip_fg_pca, valid_panns_clip_fg_pca, test_panns_clip_fg_pca,
    train_panns_embedding_raw_pca, valid_panns_embedding_raw_pca, test_panns_embedding_raw_pca,
    train_panns_embedding_bg_pca, valid_panns_embedding_bg_pca, test_panns_embedding_bg_pca,
    train_panns_embedding_fg_pca, valid_panns_embedding_fg_pca, test_panns_embedding_fg_pca,
    y_train, y_valid, y_test] = aggregated_data

In [3]:
# including panns embeddings
train_data_raw = np.concatenate((train_mps_raw_pca, train_indices_raw_pca, train_panns_embedding_raw_pca), axis = 1)
valid_data_raw = np.concatenate((valid_mps_raw_pca, valid_indices_raw_pca, valid_panns_embedding_raw_pca), axis = 1)
test_data_raw = np.concatenate((test_mps_raw_pca, test_indices_raw_pca, test_panns_embedding_raw_pca), axis = 1)

train_data_bg = np.concatenate((train_mps_bg_pca, train_indices_bg_pca, train_panns_embedding_bg_pca), axis = 1)
valid_data_bg = np.concatenate((valid_mps_bg_pca, valid_indices_bg_pca, valid_panns_embedding_bg_pca), axis = 1)
test_data_bg = np.concatenate((test_mps_bg_pca, test_indices_bg_pca, test_panns_embedding_bg_pca), axis = 1)

train_data_fg = np.concatenate((train_mps_fg_pca, train_indices_fg_pca, train_panns_embedding_fg_pca), axis = 1)
valid_data_fg = np.concatenate((valid_mps_fg_pca, valid_indices_fg_pca, valid_panns_embedding_fg_pca), axis = 1)
test_data_fg = np.concatenate((test_mps_fg_pca, test_indices_fg_pca, test_panns_embedding_fg_pca), axis = 1)

train_data_all = np.concatenate((train_data_raw, train_data_bg, train_data_fg), axis = 1)
valid_data_all = np.concatenate((valid_data_raw, valid_data_bg, valid_data_fg), axis = 1)
test_data_all = np.concatenate((test_data_raw, test_data_bg, test_data_fg), axis = 1)

In [18]:
# combining training and validating set
train_data_raw_final = np.concatenate((train_data_raw, valid_data_raw), axis=0)
train_data_bg_final = np.concatenate((train_data_bg, valid_data_bg), axis=0)
train_data_fg_final = np.concatenate((train_data_fg, valid_data_fg), axis=0)
train_data_all_final = np.concatenate((train_data_all, valid_data_all), axis=0)

y_train_final = np.concatenate((y_train, y_valid), axis=0)

In [4]:
def build_model_AUC_narrow(hp):

    model = keras.Sequential()
    model.add(layers.Input(shape=(train_data_raw.shape[1]), dtype=tf.float32))
    drop_rate = hp.Float("dr", min_value=0.2, max_value=0.5, sampling="linear")
#         if hp.Boolean("dropout"):
    model.add(layers.Dropout(rate=drop_rate))
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 5)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=10, max_value=200, step=10),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
        model.add(layers.Dropout(rate=drop_rate))
    model.add(layers.Dense(1, activation='sigmoid'))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#         loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.AUC(name='AUC')],
    )

    return model

def build_model_AUC_wide(hp):

    model = keras.Sequential()
    model.add(layers.Input(shape=(train_data_all.shape[1]), dtype=tf.float32))
    drop_rate = hp.Float("dr", min_value=0.2, max_value=0.5, sampling="linear")
#         if hp.Boolean("dropout"):
    model.add(layers.Dropout(rate=drop_rate))
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 5)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=10, max_value=500, step=50),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
        model.add(layers.Dropout(rate=drop_rate))
    model.add(layers.Dense(1, activation='sigmoid'))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.AUC(name='AUC')],
    )

    return model

In [5]:
# all
tuner_YAMNet_all = kt.BayesianOptimization(build_model_AUC_wide,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory='../model/BinaryCrossentropy_AUC',
                     project_name='kt_Bayesian_all_0313_YAMNetEmbedding')

tuner_VGG_all = kt.BayesianOptimization(build_model_AUC_wide,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory='../model/BinaryCrossentropy_AUC',
                     project_name='kt_Bayesian_all_0313_VGGEmbedding')

tuner_panns_all = kt.BayesianOptimization(build_model_AUC_wide,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory='../model/BinaryCrossentropy_AUC',
                     project_name='kt_Bayesian_all_0313_pannsEmbedding')

# raw
tuner_YAMNet_raw = kt.BayesianOptimization(build_model_AUC_narrow,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory='../model/BinaryCrossentropy_AUC',
                     project_name='kt_Bayesian_raw_0313_YAMNetEmbedding')

tuner_VGG_raw = kt.BayesianOptimization(build_model_AUC_narrow,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory='../model/BinaryCrossentropy_AUC',
                     project_name='kt_Bayesian_raw_0313_VGGEmbedding')

tuner_panns_raw = kt.BayesianOptimization(build_model_AUC_narrow,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory='../model/BinaryCrossentropy_AUC',
                     project_name='kt_Bayesian_raw_0313_pannsEmbedding')


# bg
tuner_YAMNet_bg = kt.BayesianOptimization(build_model_AUC_narrow,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory='../model/BinaryCrossentropy_AUC',
                     project_name='kt_Bayesian_bg_0313_YAMNetEmbedding')

tuner_VGG_bg = kt.BayesianOptimization(build_model_AUC_narrow,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory='../model/BinaryCrossentropy_AUC',
                     project_name='kt_Bayesian_bg_0313_VGGEmbedding')

tuner_panns_bg = kt.BayesianOptimization(build_model_AUC_narrow,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory='../model/BinaryCrossentropy_AUC',
                     project_name='kt_Bayesian_bg_0313_pannsEmbedding')


# fg
tuner_YAMNet_fg = kt.BayesianOptimization(build_model_AUC_narrow,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory='../model/BinaryCrossentropy_AUC',
                     project_name='kt_Bayesian_fg_0313_YAMNetEmbedding')

tuner_VGG_fg = kt.BayesianOptimization(build_model_AUC_narrow,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory='../model/BinaryCrossentropy_AUC',
                     project_name='kt_Bayesian_fg_0313_VGGEmbedding')

tuner_panns_fg = kt.BayesianOptimization(build_model_AUC_narrow,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory='../model/BinaryCrossentropy_AUC',
                     project_name='kt_Bayesian_fg_0313_pannsEmbedding')

INFO:tensorflow:Reloading Tuner from ../model/BinaryCrossentropy_AUC/kt_Bayesian_all_0313_YAMNetEmbedding/tuner0.json
INFO:tensorflow:Reloading Tuner from ../model/BinaryCrossentropy_AUC/kt_Bayesian_all_0313_VGGEmbedding/tuner0.json
INFO:tensorflow:Reloading Tuner from ../model/BinaryCrossentropy_AUC/kt_Bayesian_all_0313_pannsEmbedding/tuner0.json
INFO:tensorflow:Reloading Tuner from ../model/BinaryCrossentropy_AUC/kt_Bayesian_raw_0313_YAMNetEmbedding/tuner0.json


2023-03-14 15:02:07.254145: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


INFO:tensorflow:Reloading Tuner from ../model/BinaryCrossentropy_AUC/kt_Bayesian_raw_0313_VGGEmbedding/tuner0.json
INFO:tensorflow:Reloading Tuner from ../model/BinaryCrossentropy_AUC/kt_Bayesian_raw_0313_pannsEmbedding/tuner0.json
INFO:tensorflow:Reloading Tuner from ../model/BinaryCrossentropy_AUC/kt_Bayesian_bg_0313_YAMNetEmbedding/tuner0.json
INFO:tensorflow:Reloading Tuner from ../model/BinaryCrossentropy_AUC/kt_Bayesian_bg_0313_VGGEmbedding/tuner0.json
INFO:tensorflow:Reloading Tuner from ../model/BinaryCrossentropy_AUC/kt_Bayesian_bg_0313_pannsEmbedding/tuner0.json
INFO:tensorflow:Reloading Tuner from ../model/BinaryCrossentropy_AUC/kt_Bayesian_fg_0313_YAMNetEmbedding/tuner0.json
INFO:tensorflow:Reloading Tuner from ../model/BinaryCrossentropy_AUC/kt_Bayesian_fg_0313_VGGEmbedding/tuner0.json
INFO:tensorflow:Reloading Tuner from ../model/BinaryCrossentropy_AUC/kt_Bayesian_fg_0313_pannsEmbedding/tuner0.json


In [6]:
tuner_YAMNet_all.results_summary(1)
print('--------------------------------------------------')
tuner_VGG_all.results_summary(1)
print('--------------------------------------------------')
tuner_panns_all.results_summary(1)

Results summary
Results in ../model/BinaryCrossentropy_AUC/kt_Bayesian_all_0313_YAMNetEmbedding
Showing 1 best trials
Trial summary
Hyperparameters:
dr: 0.2
num_layers: 1
units_0: 460
activation: relu
lr: 0.000323865228707218
units_1: 460
units_2: 210
units_3: 210
units_4: 110
Score: 0.9748368859291077
--------------------------------------------------
Results summary
Results in ../model/BinaryCrossentropy_AUC/kt_Bayesian_all_0313_VGGEmbedding
Showing 1 best trials
Trial summary
Hyperparameters:
dr: 0.2
num_layers: 1
units_0: 460
activation: relu
lr: 0.00037306610258199647
units_1: 160
units_2: 410
units_3: 10
units_4: 260
Score: 0.911935031414032
--------------------------------------------------
Results summary
Results in ../model/BinaryCrossentropy_AUC/kt_Bayesian_all_0313_pannsEmbedding
Showing 1 best trials
Trial summary
Hyperparameters:
dr: 0.2
num_layers: 1
units_0: 460
activation: relu
lr: 0.0005271057315161746
units_1: 260
units_2: 260
units_3: 410
units_4: 260
Score: 0.977068

In [7]:
tuner_YAMNet_raw.results_summary(1)
print('--------------------------------------------------')
tuner_VGG_raw.results_summary(1)
print('--------------------------------------------------')
tuner_panns_raw.results_summary(1)

Results summary
Results in ../model/BinaryCrossentropy_AUC/kt_Bayesian_raw_0313_YAMNetEmbedding
Showing 1 best trials
Trial summary
Hyperparameters:
dr: 0.2
num_layers: 1
units_0: 200
activation: relu
lr: 0.0006924712314276848
units_1: 200
units_2: 30
units_3: 10
units_4: 200
Score: 0.9164780974388123
--------------------------------------------------
Results summary
Results in ../model/BinaryCrossentropy_AUC/kt_Bayesian_raw_0313_VGGEmbedding
Showing 1 best trials
Trial summary
Hyperparameters:
dr: 0.2
num_layers: 1
units_0: 200
activation: relu
lr: 0.00039393611757934557
units_1: 100
units_2: 100
units_3: 200
units_4: 200
Score: 0.8451542258262634
--------------------------------------------------
Results summary
Results in ../model/BinaryCrossentropy_AUC/kt_Bayesian_raw_0313_pannsEmbedding
Showing 1 best trials
Trial summary
Hyperparameters:
dr: 0.2
num_layers: 1
units_0: 200
activation: relu
lr: 0.0005966131210476074
units_1: 140
units_2: 90
units_3: 150
units_4: 10
Score: 0.9242044

In [8]:
tuner_YAMNet_bg.results_summary(1)
print('--------------------------------------------------')
tuner_VGG_bg.results_summary(1)
print('--------------------------------------------------')
tuner_panns_bg.results_summary(1)

Results summary
Results in ../model/BinaryCrossentropy_AUC/kt_Bayesian_bg_0313_YAMNetEmbedding
Showing 1 best trials
Trial summary
Hyperparameters:
dr: 0.2
num_layers: 1
units_0: 200
activation: relu
lr: 0.0008608506839418887
units_1: 60
units_2: 70
units_3: 140
units_4: 10
Score: 0.9170337319374084
--------------------------------------------------
Results summary
Results in ../model/BinaryCrossentropy_AUC/kt_Bayesian_bg_0313_VGGEmbedding
Showing 1 best trials
Trial summary
Hyperparameters:
dr: 0.2
num_layers: 1
units_0: 200
activation: relu
lr: 0.0011139535184970803
units_1: 120
units_2: 80
units_3: 100
units_4: 70
Score: 0.8423770666122437
--------------------------------------------------
Results summary
Results in ../model/BinaryCrossentropy_AUC/kt_Bayesian_bg_0313_pannsEmbedding
Showing 1 best trials
Trial summary
Hyperparameters:
dr: 0.2
num_layers: 1
units_0: 200
activation: relu
lr: 0.0006946776388015103
units_1: 10
units_2: 120
units_3: 200
units_4: 60
Score: 0.91560667753219

In [9]:
tuner_YAMNet_fg.results_summary(1)
print('--------------------------------------------------')
tuner_VGG_fg.results_summary(1)
print('--------------------------------------------------')
tuner_panns_fg.results_summary(1)

Results summary
Results in ../model/BinaryCrossentropy_AUC/kt_Bayesian_fg_0313_YAMNetEmbedding
Showing 1 best trials
Trial summary
Hyperparameters:
dr: 0.2
num_layers: 1
units_0: 200
activation: relu
lr: 0.0005887145125682295
units_1: 10
units_2: 80
units_3: 100
units_4: 170
Score: 0.9202155470848083
--------------------------------------------------
Results summary
Results in ../model/BinaryCrossentropy_AUC/kt_Bayesian_fg_0313_VGGEmbedding
Showing 1 best trials
Trial summary
Hyperparameters:
dr: 0.2
num_layers: 1
units_0: 200
activation: relu
lr: 0.0005199188510837566
units_1: 200
units_2: 150
units_3: 50
units_4: 200
Score: 0.8082389831542969
--------------------------------------------------
Results summary
Results in ../model/BinaryCrossentropy_AUC/kt_Bayesian_fg_0313_pannsEmbedding
Showing 1 best trials
Trial summary
Hyperparameters:
dr: 0.2
num_layers: 1
units_0: 200
activation: relu
lr: 0.000774873568422523
units_1: 100
units_2: 120
units_3: 110
units_4: 140
Score: 0.88962036371

In [13]:
best_hps=tuner_panns_all.get_best_hyperparameters(num_trials=1)[0]
model = tuner_panns_all.hypermodel.build(best_hps)

In [19]:
history = model.fit(train_data_all, y_train, epochs=50, validation_data=(valid_data_all, y_valid), use_multiprocessing=True)

Epoch 1/50


/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/keras/backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


373/373 [==============================] - 3s 6ms/step - loss: 0.6948 - AUC: 0.7194 - val_loss: 0.5935 - val_AUC: 0.7694
Epoch 2/50
373/373 [==============================] - 2s 6ms/step - loss: 0.5230 - AUC: 0.8233 - val_loss: 0.5828 - val_AUC: 0.7769
Epoch 3/50
373/373 [==============================] - 2s 6ms/step - loss: 0.4573 - AUC: 0.8646 - val_loss: 0.6029 - val_AUC: 0.7658
Epoch 4/50
373/373 [==============================] - 2s 7ms/step - loss: 0.4038 - AUC: 0.8971 - val_loss: 0.6075 - val_AUC: 0.7766
Epoch 5/50
373/373 [==============================] - 2s 7ms/step - loss: 0.3555 - AUC: 0.9227 - val_loss: 0.6372 - val_AUC: 0.7725
Epoch 6/50
373/373 [==============================] - 2s 6ms/step - loss: 0.3101 - AUC: 0.9424 - val_loss: 0.6827 - val_AUC: 0.7553
Epoch 7/50
373/373 [==============================] - 2s 5ms/step - loss: 0.2595 - AUC: 0.9608 - val_loss: 0.7152 - val_AUC: 0.7618
Epoch 8/50
373/373 [==============================] - 2s 6ms/step - loss: 0.2162 - AUC:

In [20]:
val_acc_per_epoch = history.history['val_AUC']

In [21]:
val_acc_per_epoch

[0.7693520188331604,
 0.7769208550453186,
 0.7658246159553528,
 0.7766436338424683,
 0.7724564075469971,
 0.7553024888038635,
 0.7618065476417542,
 0.7537552714347839,
 0.7494041323661804,
 0.75010085105896,
 0.7488839030265808,
 0.7582547664642334,
 0.7614842057228088,
 0.7447002530097961,
 0.744562566280365,
 0.7508425116539001,
 0.748630940914154,
 0.7484365701675415,
 0.7434050440788269,
 0.7431178689002991,
 0.7481602430343628,
 0.7583645582199097,
 0.755233108997345,
 0.747836172580719,
 0.7502312660217285,
 0.7624465227127075,
 0.7488362193107605,
 0.754968523979187,
 0.7645590305328369,
 0.7484860420227051,
 0.7490988969802856,
 0.7426255345344543,
 0.7497435212135315,
 0.7459927797317505,
 0.7576372623443604,
 0.7528137564659119,
 0.7513538002967834,
 0.7479658126831055,
 0.7541611790657043,
 0.7545985579490662,
 0.7492340803146362,
 0.7503653764724731,
 0.748274564743042,
 0.7357380390167236,
 0.7484375238418579,
 0.7591404914855957,
 0.7421053051948547,
 0.7478325366973877,


In [ ]:
pd.DataFrame(best_hps.get_config()['values'], index=['panns_all'])

In [ ]:
print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units_0')} and the optimal learning rate for the optimizer
is {best_hps.get('lr')}.
""")